# Topic modelling 

## LDA 

In [1]:
#pip install pyldavis


In [2]:
import numpy as np
import pandas as pd 

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
data = pd.read_csv('all_participants.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

data

,Participant
0,"Yes, thank you, Uli. The good thing is that I ..."
1,Three quick questions. One is just a follow-up...
2,"Well, good afternoon to everybody here in Lond..."
3,"Well, thank you very much. We will now begin w..."


In [4]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []

    for row in range(len(data)):
        for text in texts:
            doc = nlp(data.loc[row,"Participant"])
        
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data)
lemmatized_texts[0]

"thank good thing be have interface surrounding again somy read glass break minute meeting be kind of enough toget quality product drugstore corner be online again thank thank think mention already part major part keyfigure result think start get detailed p&l look key metric remarkable development key metric whichwere only drive also market whomsoever first strong cash flow nearly reach mark again westarte little bit slow year think mention previous occasion wasone mobility contract come contract again little bit later whichwa cash negative first quarter turnaround have already willturnaround get money back second half year cash flow think support positive message quality theresult even little bit more depressed overall growth cash flow stay verypositive right hand side asset own management % cantell development dollar yield especially yield thegovernment bond today be approach so increase be reallyremarkable again be only drive very good result also yield andespecially currency exchan

In [5]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['thank', 'good', 'thing', 'be', 'have', 'interface', 'surrounding', 'again', 'somy', 'read', 'glass', 'break', 'minute', 'meeting', 'be', 'kind', 'of', 'enough', 'toget', 'quality']


In [6]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 6), (2, 3), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1), (8, 16), (9, 4), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 8), (16, 1), (17, 27), (18, 1), (19, 1)]
absolute


In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

### Visualising data and topics

In [8]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/matheusmaciel/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/matheusmaciel/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/matheusmaciel/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/matheusmaciel/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.096849 -0.067107       1        1  85.465956
0     -0.007537  0.117963       2        1   8.716244
4     -0.064731 -0.062725       3        1   5.812597
2     -0.012847  0.005836       4        1   0.002778
1     -0.011733  0.006033       5        1   0.002426, topic_info=        Term        Freq       Total Category  logprob  loglift
56        be  125.000000  125.000000  Default  30.0000  30.0000
241     have   51.000000   51.000000  Default  29.0000  29.0000
562    think   20.000000   20.000000  Default  28.0000  28.0000
17      also   25.000000   25.000000  Default  27.0000  27.0000
308   little   24.000000   24.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
604     well    0.000073   12.433041   Topic5  -6.5641  -1.4192
439     rate    0.000073    8.724698   Topic5  -6.5690  -1.0699
15   already    0.000072    8.912424   Topic5  -6.5710  -1.0931
320    major    0.000072    5.946216   Topic5  -6.5753  -0.6928
223       go    0.000072    9.707037   Topic5  -6.5707  -1.1783

[330 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
672       2  0.668971   afternoon
673       2  1.062640  afterwards
8         1  0.931080       again
8         3  0.062072       again
15        1  0.897623     already
...     ...       ...         ...
716       2  1.062974      willbe
623       1  1.018636       write
624       1  0.960137        year
624       2  0.029095        year
624       3  0.029095        year

[199 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 3, 2])

Initial attempt at LDA topic modelling using 4 sample texts from earnings calls 

**ToDo**
- clean text to better categorise topics 

source: https://github.com/wjbmattingly/topic_modeling_textbook/blob/main/03_03_lda_model_demo.ipynb